In [2]:
from az_common_funcs import *

2023-02-03 18:23:47,532 :: [INFO] :: Logger initialized WITHOUT file handler


Things to Implement:
1. Try Different Tokenizers
2. TF-IDF Vecorization 
3. Word2Vec

In [3]:
with open("News_Category_Dataset_v3.json", mode = "r") as f:
    data = f.read()

In [7]:
df = pd.read_json("News_Category_Dataset_v3.json", lines = True)

In [10]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 209527 entries, 0 to 209526
Data columns (total 6 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   link               209527 non-null  object        
 1   headline           209527 non-null  object        
 2   category           209527 non-null  object        
 3   short_description  209527 non-null  object        
 4   authors            209527 non-null  object        
 5   date               209527 non-null  datetime64[ns]
dtypes: datetime64[ns](1), object(5)
memory usage: 9.6+ MB


,link,headline,category,short_description,authors,date
0,https://www.huffpost.com/entry/covid-boosters-...,Over 4 Million Americans Roll Up Sleeves For O...,U.S. NEWS,Health experts said it is too early to predict...,"Carla K. Johnson, AP",2022-09-23
1,https://www.huffpost.com/entry/american-airlin...,"American Airlines Flyer Charged, Banned For Li...",U.S. NEWS,He was subdued by passengers and crew when he ...,Mary Papenfuss,2022-09-23
2,https://www.huffpost.com/entry/funniest-tweets...,23 Of The Funniest Tweets About Cats And Dogs ...,COMEDY,"""Until you have a dog you don't understand wha...",Elyse Wanshel,2022-09-23
3,https://www.huffpost.com/entry/funniest-parent...,The Funniest Tweets From Parents This Week (Se...,PARENTING,"""Accidentally put grown-up toothpaste on my to...",Caroline Bologna,2022-09-23
4,https://www.huffpost.com/entry/amy-cooper-lose...,Woman Who Called Cops On Black Bird-Watcher Lo...,U.S. NEWS,Amy Cooper accused investment firm Franklin Te...,Nina Golgowski,2022-09-22


In [11]:
df.date.max(), df.date.min()

(Timestamp('2022-09-23 00:00:00'), Timestamp('2012-01-28 00:00:00'))

In [9]:
df.category.value_counts()

POLITICS          35602
WELLNESS          17945
ENTERTAINMENT     17362
TRAVEL             9900
STYLE & BEAUTY     9814
PARENTING          8791
HEALTHY LIVING     6694
QUEER VOICES       6347
FOOD & DRINK       6340
BUSINESS           5992
COMEDY             5400
SPORTS             5077
BLACK VOICES       4583
HOME & LIVING      4320
PARENTS            3955
THE WORLDPOST      3664
WEDDINGS           3653
WOMEN              3572
CRIME              3562
IMPACT             3484
DIVORCE            3426
WORLD NEWS         3299
MEDIA              2944
WEIRD NEWS         2777
GREEN              2622
WORLDPOST          2579
RELIGION           2577
STYLE              2254
SCIENCE            2206
TECH               2104
TASTE              2096
MONEY              1756
ARTS               1509
ENVIRONMENT        1444
FIFTY              1401
GOOD NEWS          1398
U.S. NEWS          1377
ARTS & CULTURE     1339
COLLEGE            1144
LATINO VOICES      1130
CULTURE & ARTS     1074
EDUCATION       

Lets take 5 categories and 5000 rows for each
1. POLITICS
1. SPORTS
1. TRAVEL
1. ENTERTAINMENT
1. BUSINESS

In [12]:
df_politics = df[df.category == "POLITICS"].reset_index(drop = True)
df_sports = df[df.category == "SPORTS"].reset_index(drop = True)
df_travel = df[df.category == "TRAVEL"].reset_index(drop = True)
df_entertainment = df[df.category == "ENTERTAINMENT"].reset_index(drop = True)
df_business = df[df.category == "BUSINESS"].reset_index(drop = True)

In [ ]:
df.politics.head()

In [13]:
import random

In [53]:
df_list = []
random.seed(123456)
for df_temp in [df_politics,df_sports,df_travel,df_entertainment,df_business]:
    r = random.sample(range(df_temp.shape[0]), 5000)
    df_temp = df_temp.copy()
    df_temp = df_temp.iloc[r]
    df_list.append(df_temp.copy())

In [57]:
df = pd.concat(df_list,ignore_index = True)

In [59]:
df = df[["short_description", "category"]]

Idea now is to vectorize the short descriptions and evaluate the embedding that are generated
Things to consider:
1. Weather to use tokenization or not. If Yes, what type of embedding 
2. Weather to remove stop words 
3. How to select top words or tokens to represents the final vector

### Not using tokenization

In [72]:
df.loc[:,"short_description_lc"] = df.short_description.str.lower()

/var/folders/ks/n4l31g9x3sj3y5jnydv95pyw0000gn/T/ipykernel_94554/4067732281.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:,"short_description_lc"] = df.short_description.str.lower()


In [95]:
import re

In [127]:
sent = "Aadil !!my is name."
pat = r"(?u)\b\w\w+\b"
def remove_punctuation(string):
#     return re.findall(pat,string.lower())
    return " ".join(re.findall(pat,string.lower()))
remove_punctuation(sent)

'aadil my is name'

In [63]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [65]:
t_vec = TfidfVectorizer(input = "content")

In [74]:
t_vec.fit(df.short_description_lc)

TfidfVectorizer()

In [89]:
t_vec.transform(["few think trump was right to fire comey."]).toarray().shape

(1, 29277)

In [81]:
df.iloc[2,2]

'few think trump was right to fire comey.'

In [93]:
t_vec.get_feature_names_out().shape

(29277,)

In [133]:
df_tfidf = pd.DataFrame(t_vec.transform(df_short_description_lc).toarray()).add_prefix("tfidf_feature_")

In [ ]:
pd.concat([df,df_tfidf], axis=1).head()

In [ ]:
import torch
from torchvision import datasets
from torchvision import transforms